# Map the competitive routes

10 most promising corridors to support (overall across operators), leave it in a separate notebook. in urban districts, strong pop signal where it could be served, which are the 10 most promising corridors.

In [1]:
import branca
import geopandas as gpd
import intake
import os
import numpy as np
import pandas as pd

import utils
from shared_utils import map_utils, geography_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0422 15:02:58.852831144     965 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
df = pd.read_parquet("./data/stripplot_trips.parquet")

In [43]:
keep_cols = [
    "calitp_itp_id", "route_id", "p50", "car_duration_hours",
    "pct_trips_competitive", "num_trips", "num_competitive",
    "bus_multiplier_spread", "bus_difference_spread",
    "caltrans_district", 
]

df2 = (df[keep_cols].drop_duplicates()
       .sort_values(["pct_trips_competitive", "bus_multiplier_spread", 
                     "num_competitive",
                     "p50"],
                    ascending=[False, True, False, True]
                   )
       .reset_index(drop=True)
      )

In [44]:
gdf = catalog.gmaps_results.read()

gdf = pd.merge(
    gdf[["calitp_itp_id", "shape_id", "route_id", "geometry"]],
    df2,
    on = ["calitp_itp_id", "route_id"],
    how = "inner",
    validate = "1:1",
)


In [45]:
# Need a numeric identifier, or branca.colormap will error
gdf = gdf.assign(
    # Try to sort/group slightly differently or else colors for same operator appear too similar
    identifier2 = gdf.apply(lambda x: np.random.randint(0, 1619), axis=1),
)


gdf = (gdf.sort_values(
    ["pct_trips_competitive", "bus_multiplier_spread", 
     "num_competitive", "p50"],
    ascending=[False, True, False, True])
       .reset_index(drop=True)
       .assign(order = gdf.index)
)

In [46]:
gdf.head(20)

,calitp_itp_id,shape_id,route_id,geometry,p50,car_duration_hours,pct_trips_competitive,num_trips,num_competitive,bus_multiplier_spread,bus_difference_spread,caltrans_district,identifier2,order
0,281,BLUE_shape,Blue Line,"POLYGON ((-122.39959 37.63493, -122.39914 37.6...",0.39,0.31,1.0,360,360,0.0,0.0,04 - Oakland,280,0
1,183,18480,4446,"POLYGON ((-118.26718 34.03173, -118.26829 34.0...",0.48,0.37,1.0,258,258,0.0,0.0,07 - Los Angeles,476,1
2,281,BUSWFG,West Field Garage,"POLYGON ((-122.40025 37.62165, -122.40017 37.6...",0.28,0.19,1.0,144,144,0.0,0.0,04 - Oakland,798,2
3,282,192160,17332,"POLYGON ((-122.48429 37.72420, -122.47940 37.7...",0.33,0.20,1.0,104,104,0.0,0.0,04 - Oakland,1173,3
4,61,shp-21-01,21,"POLYGON ((-122.06943 37.90606, -122.06911 37.9...",0.73,0.70,1.0,98,98,0.0,0.0,None,1362,4
5,61,43855,4,"POLYGON ((-122.06960 37.90602, -122.06913 37.9...",0.38,0.36,1.0,82,82,0.0,0.0,None,155,5
6,61,43857,5,"POLYGON ((-122.06891 37.90619, -122.06837 37.9...",0.25,0.17,1.0,80,80,0.0,0.0,None,1061,6
7,183,14152,870,"POLYGON ((-118.29210 33.74741, -118.29186 33.7...",0.43,0.30,1.0,68,68,0.0,0.0,07 - Los Angeles,109,7
8,226,3990004,399,"POLYGON ((-117.37326 33.18575, -117.37064 33.1...",0.88,1.14,1.0,68,68,0.0,0.0,11 - San Diego,800,8
9,350,p_1274527,16733,"POLYGON ((-122.08291 37.60908, -122.08221 37.6...",0.42,0.34,1.0,67,67,0.0,0.0,04 - Oakland,533,9


In [40]:
FIG_HEIGHT = 900
FIG_WIDTH = 700

'''
color_palette = (cp.CALITP_CATEGORY_BRIGHT_COLORS + cp.CALITP_CATEGORY_BOLD_COLORS + 
                 cp.CALITP_DIVERGING_COLORS +  cp.CALITP_SEQUENTIAL_COLORS
                )

random.seed(1234)
random.shuffle(color_palette)
'''
COLORSCALE = branca.colormap.step.Accent_08.scale(
    vmin = gdf.identifier2.min(),
    vmax=gdf.identifier2.max(),
)
#(colors = color_palette)

# Choose a quantitative column to plot, 
# otherwise using categorical/discrete colors will throw error
# TypeError: '<=' not supported between instances of 'str' and 'float'

PLOT_COL = "identifier2"
POPUP_DICT = {
    "calitp_itp_id": "ITP ID",
    "route_id": "Route ID",
    #"service_hours": "Bus Travel Time (hrs)",
    #"car_duration_hours": "Car Travel Time (hrs)",
    "shape_id": "shape_id",
}

In [41]:
m = map_utils.make_folium_choropleth_map(
    # Drop datetime, arrays cols because it's not JSON serializable
    gdf[gdf.order < 25],
    plot_col = PLOT_COL,
    popup_dict = POPUP_DICT,
    tooltip_dict = POPUP_DICT,
    colorscale = COLORSCALE,
    fig_width = FIG_WIDTH,
    fig_height = FIG_HEIGHT,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"]+1,
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Competitive Transit Routes",
    legend_name="Legend",
)

m
#m.save(f"{utils.IMG_PATH}competitive_routes.html")

In [ ]:
#m